<a href="https://colab.research.google.com/github/lucifer-lson/Construction-safety-resource-estimate-bot/blob/main/NER_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **第一步：安装依赖**

In [ ]:
# %%capture  # 这是一个 Jupyter Notebook 的魔法命令，用于隐藏命令的输出，让笔记本界面更整洁。

# 安装 unsloth 包。unsloth 是一个用于微调大型语言模型（LLM）的工具，可以让模型运行更快、占用更少内存。
!pip install unsloth

# 卸载当前已安装的 unsloth 包（如果已安装），然后从 GitHub 的源代码安装最新版本。
# 这样可以确保我们使用的是最新功能和修复。
# !pip uninstall unsloth -y &&
!pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# 安装 bitsandbytes 和 unsloth_zoo 包。
# bitsandbytes 是一个用于量化和优化模型的库，可以帮助减少模型占用的内存。
# unsloth_zoo 可能包含了一些预训练模型或其他工具，方便我们使用。
!pip install bitsandbytes unsloth_zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-a8ge4im9
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-a8ge4im9
  Resolved https://github.com/unslothai/unsloth.git to commit 05f4875aff111bf3801f6e740cc03cb7a8594c9b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# **第二步：加载预训练模型**

In [ ]:
from unsloth import FastLanguageModel  # 导入FastLanguageModel类，用来加载和使用模型
import torch  # 导入torch工具，用于处理模型的数学运算

max_seq_length = 2048  # 设置模型处理文本的最大长度，相当于给模型设置一个“最大容量”
dtype = None  # 设置数据类型，让模型自动选择最适合的精度
load_in_4bit = True  # 使用4位量化来节省内存，就像把大箱子压缩成小箱子

# 加载预训练模型，并获取tokenizer工具
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-7B",  # 指定要加载的模型名称
    max_seq_length=max_seq_length,  # 使用前面设置的最大长度
    dtype=dtype,  # 使用前面设置的数据类型
    load_in_4bit=load_in_4bit,  # 使用4位量化
    # token="hf_...",  # 如果需要访问授权模型，可以在这里填入密钥
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/100k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.52G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

## **第三步：微调前测试**

In [ ]:
prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通建筑施工领域的、具备15年以上建筑工程安全管理经验的建筑安全工程师。
请回答以下建筑安全相关问题。

### 问题：
{}

### 回答：
<think>{}"""
# 定义提示风格的字符串模板，用于格式化问题

question = "通风工在空调系统调试前参加了有限空间作业应急响应培训。请分析以上文本，给出文本中包含的安全资源实体及所属类别、工种实体及所属类别、工序实体及所属类别。"
# 定义具体的建筑安全问题

In [ ]:
FastLanguageModel.for_inference(model)
# 准备模型以进行推理

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")
# 使用 tokenizer 对格式化后的问题进行编码，并移动到 GPU

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
# 使用模型生成回答

response = tokenizer.batch_decode(outputs)
# 解码模型生成的输出为可读文本

print(response[0])
# 打印生成的回答部分

<｜begin▁of▁sentence｜>以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通建筑施工领域的、具备15年以上建筑工程安全管理经验的建筑安全工程师。
请回答以下建筑安全相关问题。

### 问题：
通风工在空调系统调试前参加了有限空间作业应急响应培训。请分析以上文本，给出文本中包含的安全资源实体及所属类别、工种实体及所属类别、工序实体及所属类别。

### 回答：
<think>
嗯，我现在需要分析用户提供的文本，找出其中的安全资源实体、工种实体和工序实体。文本是关于通风工参加空调系统调试前的应急响应培训。首先，我得明确每个实体的定义和所属类别。

安全资源实体应该包括与应急响应相关的资源，比如培训教材、手册、应急设备等。这些是为应急响应准备的资源，帮助工人在需要时使用。

工种实体就是参与培训的人员，这里提到的是通风工，所以他们属于建筑施工领域的特定工种。

工序实体则是培训的具体内容，也就是应急响应的培训，这可能涉及应急流程、设备使用等。

我需要确保每个实体都正确分类，并且解释清楚它们的作用。这样用户就能清楚了解文本中的关键要素了。
</think>

文本中包含以下安全资源实体及其所属类别、工种实体及其所属类别、工序实体及其所属类别：

1. **安全资源实体及其所属类别**：
   - **应急响应手册**：属于安全资源实体，其所属类别是应急响应相关的参考资料。
   - **应急设备**：属于安全资源实体，其所属类别是应急设备的使用和管理。

2. **工种实体及其所属类别**：
   - **通风工**：属于工种实体，其所属类别是建筑施工领域的特定工种。

3. **工序实体及其所属类别**：
   - **空调系统调试前的应急响应培训**：属于工序实体，其所属类别是安全培训的内容和形式。

通过以上分析可以看出，文本中明确了安全资源实体、工种实体和工序实体的具体内容及其所属类别，有助于进一步的安全管理与培训规划。<｜end▁of▁sentence｜>


# **第四步：加载数据集**

In [ ]:
# 定义一个用于格式化提示的多行字符串模板
train_prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通建筑施工领域的、具备15年以上建筑工程安全管理经验的建筑安全工程师。
请分析以下文本，给出文本中包含的安全资源实体及所属类别、工种实体及所属类别、工序实体及所属类别。

### 文本：
{}

### 实体及类别：

{}"""

In [ ]:
# 定义结束标记（EOS_TOKEN），用于指示文本的结束
EOS_TOKEN = tokenizer.eos_token  # 必须添加结束标记

# 导入数据集加载函数
from datasets import load_dataset
# 加载指定的数据集，选择中文语言和训练集的前500条记录
dataset = load_dataset("Lucifer118/construction-safety-resource-NER-dataset", 'default', split = "train[0:41190]", trust_remote_code=True)
# 打印数据集的列名，查看数据集中有哪些字段
print(dataset.column_names)

['text', 'entities']


In [ ]:
# 定义一个函数，用于格式化数据集中的每条记录
def formatting_prompts_func(examples):
    # 从数据集中提取问题、复杂思考过程和回答
    texts = examples["text"]
    entities_list = examples["entities"]
    formatted_texts = []  # 用于存储格式化后的文本
    # 遍历每个问题、思考过程和回答，进行格式化
    for text, entities in zip(texts, entities_list):
        # 使用字符串模板插入数据，并加上结束标记
        entity_str = ", ".join([f"{entity['name']}({entity['type']})" for entity in entities])
        # 可以根据需求修改格式化模板
        formatted_text = train_prompt_style.format(text, entity_str) + EOS_TOKEN
        formatted_texts.append(formatted_text)  # 将格式化后的文本添加到列表中
    return {
        "text": formatted_texts,  # 返回包含所有格式化文本的字典
    }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

'以下是描述任务的指令，以及提供进一步上下文的输入。\n请写出一个适当完成请求的回答。\n在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。\n\n### 指令：\n你是一位精通建筑施工领域的、具备15年以上建筑工程安全管理经验的建筑安全工程师。\n请分析以下文本，给出文本中包含的安全资源实体及所属类别、工种实体及所属类别、工序实体及所属类别。\n\n### 文本：\n【机械伤害】建筑施工机械安全事故\n\n### 实体及类别：\n\n机械伤害(安全资源::知识与技术::风险识别技术), 安全事故(安全资源::意识与心态::应急响应心理)<｜end▁of▁sentence｜>'

# **第五步：执行微调**

In [ ]:
FastLanguageModel.for_training(model)

model = FastLanguageModel.get_peft_model(
    model,  # 传入已经加载好的预训练模型
    r = 16,  # 设置 LoRA 的秩，决定添加的可训练参数数量
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",  # 指定模型中需要微调的关键模块
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,  # 设置 LoRA 的超参数，影响可训练参数的训练方式
    lora_dropout = 0,  # 设置防止过拟合的参数，这里设置为 0 表示不丢弃任何参数
    bias = "none",    # 设置是否添加偏置项，这里设置为 "none" 表示不添加
    use_gradient_checkpointing = "unsloth",  # 使用优化技术节省显存并支持更大的批量大小
    random_state = 3407,  # 设置随机种子，确保每次运行代码时模型的初始化方式相同
    use_rslora = False,  # 设置是否使用 Rank Stabilized LoRA 技术，这里设置为 False 表示不使用
    loftq_config = None,  # 设置是否使用 LoftQ 技术，这里设置为 None 表示不使用
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from trl import SFTTrainer  # 导入 SFTTrainer，用于监督式微调
from transformers import TrainingArguments  # 导入 TrainingArguments，用于设置训练参数
from unsloth import is_bfloat16_supported  # 导入函数，检查是否支持 bfloat16 数据格式

trainer = SFTTrainer(  # 创建一个 SFTTrainer 实例
    model=model,  # 传入要微调的模型
    tokenizer=tokenizer,  # 传入 tokenizer，用于处理文本数据
    train_dataset=dataset,  # 传入训练数据集
    dataset_text_field="text",  # 指定数据集中文本字段的名称
    max_seq_length=max_seq_length,  # 设置最大序列长度
    dataset_num_proc=2,  # 设置数据处理的并行进程数
    packing=False,  # 是否启用打包功能（这里设置为 False，打包可以让训练更快，但可能影响效果）
    args=TrainingArguments(  # 定义训练参数
        per_device_train_batch_size=2,  # 每个设备（如 GPU）上的批量大小
        gradient_accumulation_steps=4,  # 梯度累积步数，用于模拟大批次训练
        warmup_steps=5,  # 预热步数，训练开始时学习率逐渐增加的步数
        max_steps=75,  # 最大训练步数
        learning_rate=2e-4,  # 学习率，模型学习新知识的速度
        fp16=not is_bfloat16_supported(),  # 是否使用 fp16 格式加速训练（如果环境不支持 bfloat16）
        bf16=is_bfloat16_supported(),  # 是否使用 bfloat16 格式加速训练（如果环境支持）
        logging_steps=1,  # 每隔多少步记录一次训练日志
        optim="adamw_8bit",  # 使用的优化器，用于调整模型参数
        weight_decay=0.01,  # 权重衰减，防止模型过拟合
        lr_scheduler_type="linear",  # 学习率调度器类型，控制学习率的变化方式
        seed=3407,  # 随机种子，确保训练结果可复现
        output_dir="outputs",  # 训练结果保存的目录
        report_to="none",  # 是否将训练结果报告到外部工具（如 WandB），这里设置为不报告
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 3 | Total steps = 75
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.893500
2,4.172200
3,4.061200
4,3.838800
5,3.719300
6,3.792500
7,3.454100
8,3.001900
9,2.808100
10,2.459400


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 500 Server Error: Internal Server Error for url: https://huggingface.co/unsloth/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit/resolve/main/config.json (Request ID: Root=1-67fcc9ee-4ed8b4355a577e4246730331;426067b9-22af-449a-b068-4d92d79948f6)

Internal Error - We're working hard to fix this as soon as possible! - silently ignoring the lookup for the file config.json in unsloth/deepseek-r1-distill-qwen-7b-unsloth-bnb-4bit.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in unsloth/deepseek-r1-distill-qwen-7b-unsloth-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(


# **第六步：微调后测试**

In [ ]:
print(question) # 打印前面的问题

In [ ]:
# 将模型切换到推理模式，准备回答问题
FastLanguageModel.for_inference(model)

# 将问题转换成模型能理解的格式，并发送到 GPU 上
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 让模型根据问题生成回答，最多生成 4000 个新词
outputs = model.generate(
    input_ids=inputs.input_ids,  # 输入的数字序列
    attention_mask=inputs.attention_mask,  # 注意力遮罩，帮助模型理解哪些部分重要
    max_new_tokens=4000,  # 最多生成 4000 个新词
    use_cache=True,  # 使用缓存加速生成
)

# 将生成的回答从数字转换回文字
response = tokenizer.batch_decode(outputs)

# 打印回答
print(response[0])

# **第七步：将微调后的模型保存为** GGUF 格式

In [ ]:
# 导入 Google Colab 的 userdata 模块，用于访问用户数据
from google.colab import userdata

# 从 Google Colab 用户数据中获取 Hugging Face 的 API 令牌
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

# 将模型保存为 8 位量化格式（Q8_0）
# 这种格式文件小且运行快，适合部署到资源受限的设备
if True: model.save_pretrained_gguf("model", tokenizer,)

# 将模型保存为 16 位量化格式（f16）
# 16 位量化精度更高，但文件稍大
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# 将模型保存为 4 位量化格式（q4_k_m）
# 4 位量化文件最小，但精度可能稍低
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

# **第八步：将微调后的模型上传到** HuggingFace

In [ ]:
# 导入 Hugging Face Hub 的 create_repo 函数，用于创建一个新的模型仓库
from huggingface_hub import create_repo

# 在 Hugging Face Hub 上创建一个新的模型仓库
create_repo("Lucifer118/trained_safety_resourse_model", token=HUGGINGFACE_TOKEN, exist_ok=True)

# 将模型和分词器上传到 Hugging Face Hub 上的仓库
model.push_to_hub_gguf("Lucifer118/trained_safety_resourse_model", tokenizer, token=HUGGINGFACE_TOKEN)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 2.69 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [03:01<00:00,  5.66s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving Conard/fortunetelling/pytorch_model-00001-of-00004.bin...
Unsloth: Saving Conard/fortunetelling/pytorch_model-00002-of-00004.bin...
Unsloth: Saving Conard/fortunetelling/pytorch_model-00003-of-00004.bin...
Unsloth: Saving Conard/fortunetelling/pytorch_model-00004-of-00004.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at Conard/fortunetelling into q8_0 GGUF format.
The output location will be /content/Conard/fortunetelling/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: fortunetelling
INFO:gguf.gguf_writer:gguf: T

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Saved GGUF to https://huggingface.co/Conard/fortunetelling
